<a href="https://colab.research.google.com/github/JaeHoonKOR/NLP/blob/Colab/word2vec_tmdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#구글드라이브 마운트
from google.colab import drive
import os
drive.mount('/content/drive') #구글드라이브 마운트 위치

In [7]:
import pandas as pd
#구글 드라이브 내 다운로드 받은 csv 파일의 디렉토리 위치 설정
path_dir = '/content//sample_data' 
file = '/content//sample_data/tmdb_5000_movies.csv' #다운로드 받은 파일

df = pd.read_csv(file)
movies = df[['original_title', 'overview']]
movies = movies.dropna(axis=0)              #NaN 값이 있는 행 삭제(axis=1 >> 열 삭제)
print(movies.shape)


(4800, 2)


In [8]:
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

overview_text = movies['overview']

normalized_text = []
for string in overview_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]
print('총 샘플의 개수 : {}'.format(len(result)))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


총 샘플의 개수 : 4800


In [9]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, vector_size=100, window=2, min_count=5, workers=4, sg=0)
model.save(path_dir+'tmdb.model')

In [10]:
from gensim.models import Word2Vec

# CBOW로 Word2Vec 모델 학습
model = Word2Vec(sentences=result, vector_size=100, window=2, min_count=5, workers=4, sg=0)

# CBOW 모델을 파일로 저장
model.wv.save_word2vec_format('TMDB.cbow', binary=False)


In [11]:

print(model)

Word2Vec<vocab=5549, vector_size=100, alpha=0.025>


In [12]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from google.colab import files

# Skip-gram으로 Word2Vec 모델 학습
model = Word2Vec(sentences=result, vector_size=100, window=2, min_count=5, workers=4, sg=1)

# 모델의 KeyedVectors 객체 추출
word_vectors = model.wv

# KeyedVectors 객체를 파일로 저장
word_vectors.save_word2vec_format('TMDB.sg', binary=False)

# 파일 다운로드
print(word_vectors)



KeyedVectors<vector_size=100, 5549 keys>


In [13]:
w2v_tmdb = path_dir+'w2v_tmdb'

word_vectors = model.wv
word_vectors.save(w2v_tmdb)

wv = KeyedVectors.load(w2v_tmdb, mmap='r')
print(wv.vectors.shape)


(5549, 100)


In [14]:
result = wv.most_similar(positive=['action', 'man'], negative=['woman'])
most_similar_key, similarity = result[0]
print(f"{most_similar_key}: {similarity:.4f}")


us: 0.9105


In [15]:
result = wv.similar_by_word("man")
most_similar_key, similarity = result[0]
print(f"{most_similar_key}: {similarity:.4f}")

woman: 0.9198


In [16]:
distance = wv.distance("batman", "batman")
print(f"{distance:.1f}")


0.0


In [17]:
similarity = wv.n_similarity(['action', 'batman'], ['romance', 'comic'])
print(f"{similarity:.4f}")

###출력결과
0.9992

0.9440


0.9992